# Spark - Working with timestamps in a different timezone

In this Scala notebook we are going to work with timestamps which are in a different timezone than ours.

## Init tasks

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0`

import $ivy.$                                  

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

In [4]:
val spark = SparkSession.builder().appName("Spark").master("local[*]").getOrCreate()

spark: SparkSession = org.apache.spark.sql.SparkSession@677af6a1

In [5]:
import spark.implicits._

import spark.implicits._

## Timezones and timestamp functions

Before we begin we need to take a few things into consideration:

- [UTC is indicated by the letter Z](https://www.w3.org/TR/timezone/) (for example 08:00:00**Z**)
- What is **Daylight Saving Time** ([DST](https://www.timeanddate.com/time/dst/))
- [List of Time zone ID](https://www.ibm.com/support/knowledgecenter/en/SSAW57_8.5.5/com.ibm.websphere.nd.multiplatform.doc/ae/rrun_svr_timezones.html)
- [Timestamp functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$): We have to talke a look at the description of these functions: `to_timestamp`, `to_utc_timestamp` and `from_utc_timestamp`.

For example, we can use `to_utc_timestamp` to read a timestamp from UTC+1 timezone and show it in UTC. By using `from_utc_timestamp` we can read that same UTC timestamp and show it in UTC+1.

Now, let's start by seeing the timezone in where we are:

In [6]:
spark.conf.get("spark.sql.session.timeZone")

res5: String = "Europe/Paris"

`Europe/Paris` is UTC+1 (with no DST).

## Casting timestamp for a given timezone
Then, let's create a dataframe with some timestamps in string type:

In [7]:
val df = Seq(                     // Assuming timestamps are in the same timezone as ours: UTC+1
  (1, "2018-03-24 10:10:10.100"), // No DST (UTC+1) -- to UTC must be 9h
  (2, "2018-03-24 10:10:10.100Z"), // No DST (UTC) -- to UTC must be 10h
  (3, "2018-03-25 10:10:10.100"), // DST (UTC+2) -- to UTC must be 8h
  (4, "2018-03-25 10:10:10.100Z"), // DST (UTC) -- to UTC must be 10h
  (5, "2018-11-21 11:11:11.111"), // No DST (UTC+1) -- to UTC must be 10h
  (6, "2018-11-21 11:11:11.111Z"), // No DST (UTC) -- to UTC must be 11h
  (7, "2018-12-22 22:22:22.222") // No DST (UTC+1) -- to UTC must be 21h
).toDF("id", "myTS")

df: org.apache.spark.sql.package.DataFrame = [id: int, myTS: string]

Let's add 2 new columns in order to see what we are doing:
 - `myTS_UTC` will have the original timestamp converted to UTC timestamp type.
 - `myTS_UTC+1` will use the previous column in order to show it in UTC+1.

In [8]:
df
   .withColumn("myTS_UTC", to_utc_timestamp($"myTS", "Europe/Paris"))
   .withColumn("myTS_UTC+1", from_utc_timestamp($"myTS_UTC", "Europe/Paris"))
    .show(false)

+---+------------------------+-----------------------+-----------------------+
|id |myTS                    |myTS_UTC               |myTS_UTC+1             |
+---+------------------------+-----------------------+-----------------------+
|1  |2018-03-24 10:10:10.100 |2018-03-24 09:10:10.1  |2018-03-24 10:10:10.1  |
|2  |2018-03-24 10:10:10.100Z|2018-03-24 10:10:10.1  |2018-03-24 11:10:10.1  |
|3  |2018-03-25 10:10:10.100 |2018-03-25 08:10:10.1  |2018-03-25 10:10:10.1  |
|4  |2018-03-25 10:10:10.100Z|2018-03-25 10:10:10.1  |2018-03-25 12:10:10.1  |
|5  |2018-11-21 11:11:11.111 |2018-11-21 10:11:11.111|2018-11-21 11:11:11.111|
|6  |2018-11-21 11:11:11.111Z|2018-11-21 11:11:11.111|2018-11-21 12:11:11.111|
|7  |2018-12-22 22:22:22.222 |2018-12-22 21:22:22.222|2018-12-22 22:22:22.222|
+---+------------------------+-----------------------+-----------------------+



The column `myTS` is the original timestamp in string type, `myTS_UTC` represents the timestamp in UTC and `myTS_UTC+1` get that timestamp and shows the timestamp in UTC+1.

As we can see, **it also applies the DST correctly**: on 25th March it begins [DST in 2018](https://www.timeanddate.com/time/dst/2018.html) and because at that time we are in UTC+2, **the difference is 2 hours**: `2018-03-25 10:10:10.1` UTC timestamp vs `2018-03-25 12:10:10.1` UTC+1 timestamp.


## How our timezone affects the timestamps with Z
Now, we will do the same test but setting our timezone to UTC+3 in order to see how the timestamps with Z (UTC) differs respect the one with no Z.

In [9]:
spark.conf.set("spark.sql.session.timeZone", "Europe/Moscow") // UTC+3

And now run the same code as before:

In [10]:
df
   .withColumn("myTS_UTC", to_utc_timestamp($"myTS", "Europe/Paris"))
   .withColumn("myTS_UTC+1", from_utc_timestamp($"myTS_UTC", "Europe/Paris"))
    .show(false)

+---+------------------------+-----------------------+-----------------------+
|id |myTS                    |myTS_UTC               |myTS_UTC+1             |
+---+------------------------+-----------------------+-----------------------+
|1  |2018-03-24 10:10:10.100 |2018-03-24 09:10:10.1  |2018-03-24 10:10:10.1  |
|2  |2018-03-24 10:10:10.100Z|2018-03-24 12:10:10.1  |2018-03-24 13:10:10.1  |
|3  |2018-03-25 10:10:10.100 |2018-03-25 08:10:10.1  |2018-03-25 10:10:10.1  |
|4  |2018-03-25 10:10:10.100Z|2018-03-25 11:10:10.1  |2018-03-25 13:10:10.1  |
|5  |2018-11-21 11:11:11.111 |2018-11-21 10:11:11.111|2018-11-21 11:11:11.111|
|6  |2018-11-21 11:11:11.111Z|2018-11-21 13:11:11.111|2018-11-21 14:11:11.111|
|7  |2018-12-22 22:22:22.222 |2018-12-22 21:22:22.222|2018-12-22 22:22:22.222|
+---+------------------------+-----------------------+-----------------------+



What happened?

**The normal timestamp (the one with no Z)**, when converted to UTC, it does not care about configured TZ, it only uses the one indicated in the function parameter.

**The UTC timestamp (the one with Z)** takes into account our current TZ (Moscow UTC+3) and because of this, when converted to UTC, it calculates based on the fact that timestamp is actually 13h (Moscow UTC+3). `to_utc_timestamp` function gets the 13h interpreted as a Paris TZ (UTC+1), so final conversion is the result of subtracting that Paris UTC+1 hour from the timestamp.

The exception here is 25th March: in this case the difference is only 1h because [in Moscow there was no DST in 2018](https://www.timeanddate.com/time/change/russia/moscow).

For a clearer information, here we can see how the Z takes into consideration our timezone whereas the timestamp with no Z has no change:

![Timestamps and timezones](../files/timestamps_timezones.png "Timestamps and timezones")

